In [1]:
dataset_folder_name = 'TrainFiles/AgeChange/FourChange'
testset_folder_name_W = 'TestFiles/White'
testset_folder_name_B = 'TestFiles/Black'

TRAIN_TEST_SPLIT = 0.7
IM_WIDTH = IM_HEIGHT = 198

dataset_dict = {
    'race_id': {
        0: 'white', 
        1: 'black', 
        2: 'asian', 
        3: 'indian', 
        4: 'others'
    },
    'gender_id': {
        0: 'male',
        1: 'female'
    }
}

dataset_dict['gender_alias'] = dict((g, i) for i, g in dataset_dict['gender_id'].items())

import numpy as np 
import pandas as pd
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

# Parsing

In [2]:
def parse_dataset(dataset_path, ext='jpg'):
    """
    Used to extract information about our dataset. It does iterate over all images and return a DataFrame with
    the data (age, gender and sex) of all files.
    """
    def parse_info_from_file(path):
        """
        Parse information from a single file
        """
        try:
            filename = os.path.split(path)[1]
            filename = os.path.splitext(filename)[0]
            age, gender, race, _ = filename.split('_')

            return int(age)
        except Exception as ex:
            return None, None, None
        
    files = glob.glob(os.path.join(dataset_path, "*.%s" % ext))
    random.shuffle(files)
    
    records = []
    for file in files:
        info = parse_info_from_file(file)
        records.append(info)
        
    df = pd.DataFrame(records)
    df['file'] = files
    df.columns = ['age', 'file']
    df = df.dropna()
    
    return df

In [3]:
df = parse_dataset(dataset_folder_name)
testset_W = parse_dataset(testset_folder_name_W)
testset_B = parse_dataset(testset_folder_name_B)
df.head()

,age,file
0,43,TrainFiles/AgeChange/FourChange/43_0_0_2017010...
1,11,TrainFiles/AgeChange/FourChange/11_0_0_2017011...
2,100,TrainFiles/AgeChange/FourChange/100_1_0_201701...
3,25,TrainFiles/AgeChange/FourChange/25_0_1_2017012...
4,5,TrainFiles/AgeChange/FourChange/5_1_0_20170109...


# Makes Data Generator For Trainset

In [4]:
from keras.utils import to_categorical
from PIL import Image


class UtkFaceDataGenerator():
    """
    Data generator for the UTKFace dataset. This class should be used when training our Keras multi-output model.
    """
    def __init__(self, df):
        self.df = df
        
    def generate_split_indexes(self, SPLIT):
        p = np.random.permutation(len(self.df))
        train_up_to = int(len(self.df) * SPLIT)
        train_idx = p[:train_up_to]
        test_idx = p[train_up_to:]

        train_up_to = int(train_up_to * SPLIT)
        train_idx, valid_idx = train_idx[:train_up_to], train_idx[train_up_to:]
        
        # converts alias to id
        self.max_age = self.df['age'].max()
        
        return train_idx, valid_idx, test_idx
    
    def preprocess_image(self, img_path):
        """
        Used to perform some minor preprocessing on the image before inputting into the network.
        """
        im = Image.open(img_path)
        im = im.resize((IM_WIDTH, IM_HEIGHT))
        im = np.array(im) / 255.0
        
        return im
        
    def generate_images(self, image_idx, is_training, batch_size=16):
        """
        Used to generate a batch with images when training/testing/validating our Keras model.
        """
        
        # arrays to store our batched data
        images, ages = [], []
        while True:
            for idx in image_idx:
                person = self.df.iloc[idx]
                
                age = person['age']
                file = person['file']
                
                im = self.preprocess_image(file)
                ages.append(age / self.max_age)
                images.append(im)
                
                # yielding condition
                if len(images) >= batch_size:
                    yield np.array(images), [np.array(ages)]
                    images, ages = [], []
                    
            if not is_training:
                break
                
data_generator = UtkFaceDataGenerator(df)
train_idx, valid_idx, test_idx = data_generator.generate_split_indexes(TRAIN_TEST_SPLIT)

Using TensorFlow backend.


# Data Generator TestSet

In [5]:
TEST_DATA_SPLIT = 0
#White
test_data_generator_W = UtkFaceDataGenerator(testset_W)
train_idx_test_W, valid_idx_test_W, test_idx_test_W = test_data_generator_W.generate_split_indexes(TEST_DATA_SPLIT)
#Black
test_data_generator_B = UtkFaceDataGenerator(testset_B)
train_idx_test_B, valid_idx_test_B, test_idx_test_B = test_data_generator_B.generate_split_indexes(TEST_DATA_SPLIT)







# # Making Model

In [6]:
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Lambda
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
import tensorflow as tf

class UtkMultiOutputModel():
    """
    Used to generate our multi-output model. This CNN contains three branches, one for age, other for 
    sex and another for race. Each branch contains a sequence of Convolutional Layers that is defined
    on the make_default_hidden_layers method.
    """
    def make_default_hidden_layers(self, inputs):
        """
        Used to generate a default set of hidden layers. The structure used in this network is defined as:
        
        Conv2D -> BatchNormalization -> Pooling -> Dropout
        """
        x = Conv2D(16, (3, 3), padding="same")(inputs)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.25)(x)

        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)

        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)

        return x

    def build_race_branch(self, inputs, num_races):
        """
        Used to build the race branch of our face recognition network.
        This branch is composed of three Conv -> BN -> Pool -> Dropout blocks, 
        followed by the Dense output layer.
        """
        x = self.make_default_hidden_layers(inputs)

        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(num_races)(x)
        x = Activation("softmax", name="race_output")(x)

        return x

    def build_gender_branch(self, inputs, num_genders=2):
        """
        Used to build the gender branch of our face recognition network.
        This branch is composed of three Conv -> BN -> Pool -> Dropout blocks, 
        followed by the Dense output layer.
        """
        x = Lambda(lambda c: tf.image.rgb_to_grayscale(c))(inputs)

        x = self.make_default_hidden_layers(inputs)

        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(num_genders)(x)
        x = Activation("sigmoid", name="gender_output")(x)

        return x

    def build_age_branch(self, inputs):   
        """
        Used to build the age branch of our face recognition network.
        This branch is composed of three Conv -> BN -> Pool -> Dropout blocks, 
        followed by the Dense output layer.

        """
        x = self.make_default_hidden_layers(inputs)

        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(1)(x)
        x = Activation("linear", name="age_output")(x)

        return x

    def assemble_full_model(self, width, height):
        """
        Used to assemble our multi-output model CNN.
        """
        input_shape = (height, width, 3)

        inputs = Input(shape=input_shape)

        age_branch = self.build_age_branch(inputs)

        model = Model(inputs=inputs,
                     outputs = [age_branch],
                     name="face_net")

        return model
    
model = UtkMultiOutputModel().assemble_full_model(IM_WIDTH, IM_HEIGHT)













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
'''%matplotlib inline

from keras.utils import plot_model
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

plot_model(model, to_file='model.png')
img = mpimg.imread('model.png')

plt.figure(figsize=(40, 30))
plt.imshow(img)'''

"%matplotlib inline\n\nfrom keras.utils import plot_model\nimport matplotlib.image as mpimg\nimport matplotlib.pyplot as plt\n\nplot_model(model, to_file='model.png')\nimg = mpimg.imread('model.png')\n\nplt.figure(figsize=(40, 30))\nplt.imshow(img)"

# Training

In [8]:
from keras.optimizers import Adam

init_lr = 1e-4
epochs = 100

opt = Adam(lr=init_lr, decay=init_lr / epochs)

model.compile(optimizer=opt, 
              loss={
                  'age_output': 'mse'},
              loss_weights={ 
                  'age_output': 4.},
              metrics={
                  'age_output': 'mae'})

valid_batch_size = 32
print(len(valid_idx))
print(len(valid_idx)//valid_batch_size)


2206
68


And now let's train our model with a batch size of 32 for both valid and train sets. We will be using a ModelCheckpoint callback in order to save the model on disk at the end of each epoch.

In [9]:
from keras.callbacks import ModelCheckpoint

batch_size = 32
valid_batch_size = 32
train_gen = data_generator.generate_images(train_idx, is_training=True, batch_size=batch_size)
valid_gen = data_generator.generate_images(valid_idx, is_training=True, batch_size=valid_batch_size)

callbacks = [
    ModelCheckpoint("./model_checkpoint", monitor='val_loss')
]

history = model.fit_generator(train_gen,
                    steps_per_epoch=len(train_idx)//batch_size,
                    epochs=epochs,
                    callbacks=callbacks,
                    validation_data=valid_gen,
                    validation_steps=len(valid_idx)//valid_batch_size)



Epoch 1/100
160/160 [==============================] - 20s 123ms/step - loss: 14.2040 - mean_absolute_error: 1.4659 - val_loss: 6.0453 - val_mean_absolute_error: 0.9496
Epoch 2/100
160/160 [==============================] - 13s 82ms/step - loss: 10.0921 - mean_absolute_error: 1.2419 - val_loss: 11.9339 - val_mean_absolute_error: 1.2423
Epoch 3/100
160/160 [==============================] - 14s 85ms/step - loss: 7.8839 - mean_absolute_error: 1.0879 - val_loss: 28.8032 - val_mean_absolute_error: 1.8186
Epoch 4/100
160/160 [==============================] - 13s 83ms/step - loss: 6.5794 - mean_absolute_error: 1.0035 - val_loss: 93.9610 - val_mean_absolute_error: 3.1461
Epoch 5/100
160/160 [==============================] - 13s 83ms/step - loss: 6.1284 - mean_absolute_error: 0.9566 - val_loss: 154.8302 - val_mean_absolute_error: 4.0734
Epoch 6/100
160/160 [==============================] - 13s 83ms/step - loss: 5.4236 - mean_absolute_error: 0.8887 - val_loss: 184.3596 - val_mean_absolute_

Epoch 47/100
160/160 [==============================] - 13s 84ms/step - loss: 0.0767 - mean_absolute_error: 0.1085 - val_loss: 9.5227 - val_mean_absolute_error: 0.5453
Epoch 48/100
160/160 [==============================] - 13s 83ms/step - loss: 0.0750 - mean_absolute_error: 0.1069 - val_loss: 8.5806 - val_mean_absolute_error: 0.5165
Epoch 49/100
160/160 [==============================] - 13s 83ms/step - loss: 0.0702 - mean_absolute_error: 0.1035 - val_loss: 5.9974 - val_mean_absolute_error: 0.4269
Epoch 50/100
160/160 [==============================] - 13s 84ms/step - loss: 0.0704 - mean_absolute_error: 0.1035 - val_loss: 6.1288 - val_mean_absolute_error: 0.4363
Epoch 51/100
160/160 [==============================] - 14s 85ms/step - loss: 0.0697 - mean_absolute_error: 0.1033 - val_loss: 6.4668 - val_mean_absolute_error: 0.4483
Epoch 52/100
160/160 [==============================] - 13s 84ms/step - loss: 0.0667 - mean_absolute_error: 0.1006 - val_loss: 9.8695 - val_mean_absolute_error:

Epoch 96/100
160/160 [==============================] - 13s 83ms/step - loss: 0.0370 - mean_absolute_error: 0.0746 - val_loss: 0.7266 - val_mean_absolute_error: 0.1225
Epoch 97/100
160/160 [==============================] - 13s 83ms/step - loss: 0.0369 - mean_absolute_error: 0.0751 - val_loss: 0.7501 - val_mean_absolute_error: 0.1213
Epoch 98/100
160/160 [==============================] - 13s 84ms/step - loss: 0.0359 - mean_absolute_error: 0.0735 - val_loss: 0.9084 - val_mean_absolute_error: 0.1262
Epoch 99/100
160/160 [==============================] - 13s 83ms/step - loss: 0.0355 - mean_absolute_error: 0.0742 - val_loss: 0.4970 - val_mean_absolute_error: 0.1116
Epoch 100/100
160/160 [==============================] - 13s 83ms/step - loss: 0.0354 - mean_absolute_error: 0.0734 - val_loss: 0.4643 - val_mean_absolute_error: 0.1048


In [10]:
import plotly.graph_objects as go
from IPython.display import display, Image

### Overall loss

# Testing

In [11]:
from PIL import Image
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
test_batch_size = 128


# WHITE

In [12]:
test_generator = test_data_generator_W.generate_images(test_idx_test_W, is_training=False, batch_size=test_batch_size)
age_pred = model.predict_generator(test_generator, steps=len(test_idx_test_W)//test_batch_size)



test_generator = test_data_generator_W.generate_images(test_idx_test_W, is_training=False, batch_size=test_batch_size)
samples = 0
images, age_true = [], []
for test_batch in test_generator:
    image = test_batch[0]
    labels = test_batch[1]
    
    images.extend(image)
    age_true.extend(labels[0])

age_true = np.array(age_true)

age_true = age_true * data_generator.max_age
age_pred = age_pred * data_generator.max_age


from sklearn.metrics import classification_report

'''cr_age = classification_report(age_true, age_pred, target_names=dataset_dict['gender_alias'].keys())
print(cr_age)'''
print('R2 score for age: ', r2_score(age_true, age_pred))
print('Deviance Score for age:', explained_variance_score(age_true, age_pred))

R2 score for age:  -4.883476167077703
Deviance Score for age: -4.883354119204407


# BLACK

In [13]:
test_generator = test_data_generator_B.generate_images(test_idx_test_B, is_training=False, batch_size=test_batch_size)
age_pred = model.predict_generator(test_generator, steps=len(test_idx_test_B)//test_batch_size)



test_generator = test_data_generator_B.generate_images(test_idx_test_B, is_training=False, batch_size=test_batch_size)
samples = 0
images, age_true = [], []
for test_batch in test_generator:
    image = test_batch[0]
    labels = test_batch[1]
    
    images.extend(image)
    age_true.extend(labels[0])

age_true = np.array(age_true)

age_true = age_true * (data_generator.max_age)
age_pred = age_pred * (data_generator.max_age)


from sklearn.metrics import classification_report

'''cr_age = classification_report(age_true, age_pred, target_names=dataset_dict['gender_alias'].keys())
print(cr_age)'''
print('R2 score for age: ', r2_score(age_true, age_pred))




R2 score for age:  -16.41406893345584


In [14]:
print(age_true)
print(age_pred)

[ 33.65656566  43.27272727  69.71717172  38.46464646  78.13131313
  31.25252525  37.26262626  37.26262626  36.06060606  31.25252525
  56.49494949  56.49494949  34.85858586  37.26262626  40.86868687
  33.65656566  33.65656566  42.07070707  33.65656566  54.09090909
  63.70707071  46.87878788  42.07070707  31.25252525  36.06060606
   3.60606061  48.08080808  81.73737374  32.45454545  16.82828283
  80.53535354  33.65656566  18.03030303  34.85858586   3.60606061
  31.25252525 108.18181818  31.25252525  34.85858586  60.1010101
  37.26262626  21.63636364  64.90909091   2.4040404   33.65656566
  32.45454545  48.08080808  33.65656566  37.26262626  45.67676768
  30.05050505  84.14141414  44.47474747  48.08080808  48.08080808
  28.84848485  40.86868687  37.26262626  31.25252525  40.86868687
  37.26262626  31.25252525  33.65656566  45.67676768  33.65656566
  37.26262626  42.07070707  32.45454545  30.05050505  32.45454545
  26.44444444  31.25252525  31.25252525  48.08080808  38.46464646
  18.030303

In [15]:
print('Deviance Score for age:', explained_variance_score(age_true, age_pred))

Deviance Score for age: -16.175936770991388
